In [ ]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display

import argparse
import time

import copy

In [ ]:
torch.manual_seed(1234)  #Reproducibility

GPU = True
device_idx = 0
if GPU:
    device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda:0


In [ ]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = int((self.position + 1) % self.capacity)  # as a ring buffer
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch)) # stack for each element
        ''' 
        the * serves as unpack: sum(a,b) <=> batch=(a,b), sum(*batch) ;
        zip: a=[1,2], b=[2,3], zip(a,b) => [(1, 2), (2, 3)] ;
        the map serves as mapping the function on each list element: map(square, [2,3]) => [4,9] ;
        np.stack((1,2)) => array([1, 2])
        '''
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

In [ ]:
class NormalizedActions(gym.ActionWrapper):
    def action(self, action):
        low  = self.action_space.low
        high = self.action_space.high
        
        action = low + (action + 1.0) * 0.5 * (high - low)
        action = np.clip(action, low, high)
        
        return action

    def _reverse_action(self, action):
        low  = self.action_space.low
        high = self.action_space.high
        
        action = 2 * (action - low) / (high - low) - 1
        action = np.clip(action, low, high)
        
        return action


In [ ]:
class QNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3):
        super(QNetwork, self).__init__()
        
        self.linear1 = nn.Linear(num_inputs + num_actions, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, hidden_size)
        self.linear4 = nn.Linear(hidden_size, 1)
        
        self.linear4.weight.data.uniform_(-init_w, init_w)
        self.linear4.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state, action):
        x = torch.cat([state, action], 1) # the dim 0 is number of samples
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = self.linear4(x)
        return x

In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, action_range=1., init_w=3e-3, log_std_min=-20, log_std_max=2):
        super(PolicyNetwork, self).__init__()
        
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max
        
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, hidden_size)
        self.linear4 = nn.Linear(hidden_size, hidden_size)

        self.mean_linear = nn.Linear(hidden_size, num_actions)
        self.mean_linear.weight.data.uniform_(-init_w, init_w)
        self.mean_linear.bias.data.uniform_(-init_w, init_w)
        
        self.log_std_linear = nn.Linear(hidden_size, num_actions)
        self.log_std_linear.weight.data.uniform_(-init_w, init_w)
        self.log_std_linear.bias.data.uniform_(-init_w, init_w)

        self.action_range = action_range
        self.num_actions = num_actions

        
    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))

        mean  = torch.tanh(self.mean_linear(x))
        # mean = F.leaky_relu(self.mean_linear(x))
        # mean = torch.clamp(mean, -30, 30)

        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max) # clip the log_std into reasonable range
        
        return mean, log_std
    
    def evaluate(self, state, deterministic, eval_noise_scale, epsilon=1e-6):
        '''
        generate action with state as input wrt the policy network, for calculating gradients
        '''
        mean, log_std = self.forward(state)
        std = log_std.exp() # no clip in evaluation, clip affects gradients flow
        
        normal = Normal(0, 1)
        z      = normal.sample() 
        action_0 = torch.tanh(mean + std*z.to(device)) # TanhNormal distribution as actions; reparameterization trick
        action = self.action_range*mean if deterministic else self.action_range*action_0
        log_prob = Normal(mean, std).log_prob(mean+ std*z.to(device)) - torch.log(1. - action_0.pow(2) + epsilon) -  np.log(self.action_range)
        # both dims of normal.log_prob and -log(1-a**2) are (N,dim_of_action); 
        # the Normal.log_prob outputs the same dim of input features instead of 1 dim probability, 
        # needs sum up across the features dim to get 1 dim prob; or else use Multivariate Normal.
        log_prob = log_prob.sum(dim=1, keepdim=True)
        ''' add noise '''
        eval_noise_clip = 2*eval_noise_scale
        noise = normal.sample(action.shape) * eval_noise_scale
        noise = torch.clamp(
        noise,
        -eval_noise_clip,
        eval_noise_clip)
        action = action + noise.to(device)

        return action, log_prob, z, mean, log_std
        
    
    def get_action(self, state, deterministic, explore_noise_scale):
        '''
        generate action for interaction with env
        '''
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        mean, log_std = self.forward(state)
        std = log_std.exp()
        
        normal = Normal(0, 1)
        z      = normal.sample().to(device)
        
        action = mean.detach().cpu().numpy()[0] if deterministic else torch.tanh(mean + std*z).detach().cpu().numpy()[0]

        ''' add noise '''
        noise = normal.sample(action.shape) * explore_noise_scale
        action = self.action_range*action + noise.numpy()

        return action

    def sample_action(self,):
        a=torch.FloatTensor(self.num_actions).uniform_(-1, 1)
        return self.action_range*a.numpy()

In [ ]:
class TD3_Trainer():
    def __init__(self, replay_buffer, hidden_dim, action_range, policy_target_update_interval=1):
        self.replay_buffer = replay_buffer


        self.q_net1 = QNetwork(state_dim, action_dim, hidden_dim).to(device)
        self.q_net2 = QNetwork(state_dim, action_dim, hidden_dim).to(device)
        self.target_q_net1 = QNetwork(state_dim, action_dim, hidden_dim).to(device)
        self.target_q_net2 = QNetwork(state_dim, action_dim, hidden_dim).to(device)
        self.policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim, action_range).to(device)
        self.target_policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim, action_range).to(device)
        print('Q Network (1,2): ', self.q_net1)
        print('Policy Network: ', self.policy_net)

        self.target_q_net1 = self.target_ini(self.q_net1, self.target_q_net1)
        self.target_q_net2 = self.target_ini(self.q_net2, self.target_q_net2)
        self.target_policy_net = self.target_ini(self.policy_net, self.target_policy_net)
        

        q_lr = 3e-4
        policy_lr = 3e-4
        self.update_cnt = 0
        self.policy_target_update_interval = policy_target_update_interval

        self.q_optimizer1 = optim.Adam(self.q_net1.parameters(), lr=q_lr)
        self.q_optimizer2 = optim.Adam(self.q_net2.parameters(), lr=q_lr)
        self.policy_optimizer = optim.Adam(self.policy_net.parameters(), lr=policy_lr)
    
    def target_ini(self, net, target_net):
        for target_param, param in zip(target_net.parameters(), net.parameters()):
            target_param.data.copy_(param.data)
        return target_net

    def target_soft_update(self, net, target_net, soft_tau):
    # Soft update the target net
        for target_param, param in zip(target_net.parameters(), net.parameters()):
            target_param.data.copy_(  # copy data value into target parameters
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

        return target_net
    
    def update(self, batch_size, deterministic, eval_noise_scale, reward_scale=10., gamma=0.9,soft_tau=1e-2):
        loss = dict(q_loss = None, policy_loss = None)
        state, action, reward, next_state, done = self.replay_buffer.sample(batch_size)
        # print('sample:', state, action,  reward, done)

        state      = torch.FloatTensor(state).to(device)
        next_state = torch.FloatTensor(next_state).to(device)
        action     = torch.FloatTensor(action).to(device)
        reward     = torch.FloatTensor(reward).unsqueeze(1).to(device)  # reward is single value, unsqueeze() to add one dim to be [reward] at the sample dim;
        done       = torch.FloatTensor(np.float32(done)).unsqueeze(1).to(device)

        predicted_q_value1 = self.q_net1(state, action)
        predicted_q_value2 = self.q_net2(state, action)
        new_action, log_prob, z, mean, log_std = self.policy_net.evaluate(state, deterministic, eval_noise_scale=0.0)  # no noise, deterministic policy gradients
        new_next_action, _, _, _, _ = self.target_policy_net.evaluate(next_state, deterministic, eval_noise_scale=eval_noise_scale) # clipped normal noise

        reward = reward_scale * (reward - reward.mean(dim=0)) / (reward.std(dim=0) + 1e-6) # normalize with batch mean and std; plus a small number to prevent numerical problem

    # Training Q Function
        target_q_min = torch.min(self.target_q_net1(next_state, new_next_action),self.target_q_net2(next_state, new_next_action))

        target_q_value = reward + (1 - done) * gamma * target_q_min # if done==1, only reward

        q_value_loss1 = ((predicted_q_value1 - target_q_value.detach())**2).mean()  # detach: no gradients for the variable
        q_value_loss2 = ((predicted_q_value2 - target_q_value.detach())**2).mean()
        net_q_loss = (q_value_loss1 + q_value_loss2).mean()
        loss['q_loss'] = net_q_loss.detach().cpu().numpy()

        self.q_optimizer1.zero_grad()
        q_value_loss1.backward()
        self.q_optimizer1.step()
        self.q_optimizer2.zero_grad()
        q_value_loss2.backward()
        self.q_optimizer2.step()

        if self.update_cnt%self.policy_target_update_interval==0:

        # Training Policy Function
            ''' implementation 1 '''
            # predicted_new_q_value = torch.min(self.q_net1(state, new_action),self.q_net2(state, new_action))
            ''' implementation 2 '''
            predicted_new_q_value = self.q_net1(state, new_action)

            policy_loss = - predicted_new_q_value.mean()

            p_loss = policy_loss.clone()
            loss['policy_loss'] = p_loss.detach().cpu().numpy()

            self.policy_optimizer.zero_grad()
            policy_loss.backward()
            self.policy_optimizer.step()
        
        # Soft update the target nets
            self.target_q_net1=self.target_soft_update(self.q_net1, self.target_q_net1, soft_tau)
            self.target_q_net2=self.target_soft_update(self.q_net2, self.target_q_net2, soft_tau)
            self.target_policy_net=self.target_soft_update(self.policy_net, self.target_policy_net, soft_tau)

        self.update_cnt+=1

        return loss

In [ ]:
env = NormalizedActions(gym.make('Pendulum-v0'))
action_dim = env.action_space.shape[0]
state_dim  = env.observation_space.shape[0]
action_range=1.

In [ ]:
replay_buffer_size = 5e5
replay_buffer = ReplayBuffer(replay_buffer_size)


# hyper-parameters for RL training
max_episodes  = 1000
max_steps   = 100  # Pendulum needs 150 steps per episode to learn well, cannot handle 20
frame_idx   = 0
batch_size  = 128
explore_steps = 0  # for random action sampling in the beginning of training
update_itr = 1
hidden_dim = 512
policy_target_update_interval = 3 # delayed update for the policy network and target networks
DETERMINISTIC=False  # DDPG: deterministic policy gradient
explore_noise_scale = 0.5  # 0.5 noise is required for Pendulum-v0, 0.1 noise for HalfCheetah-v2
eval_noise_scale = 0.5
reward_scale = 1.
rewards     = []
q_loss = []
policy_loss = []

In [ ]:
td3_trainer=TD3_Trainer(replay_buffer, hidden_dim=hidden_dim, policy_target_update_interval=policy_target_update_interval, action_range=action_range )


Q Network (1,2):  QNetwork(
  (linear1): Linear(in_features=4, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=512, bias=True)
  (linear4): Linear(in_features=512, out_features=1, bias=True)
)
Policy Network:  PolicyNetwork(
  (linear1): Linear(in_features=3, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=512, bias=True)
  (linear4): Linear(in_features=512, out_features=512, bias=True)
  (mean_linear): Linear(in_features=512, out_features=1, bias=True)
  (log_std_linear): Linear(in_features=512, out_features=1, bias=True)
)


In [ ]:
for eps in range(max_episodes):
  state =  env.reset()
  episode_reward = 0
  episode_q_loss = []
  episode_policy_loss = []
            
  for step in range(max_steps):
    if frame_idx > explore_steps:
      action = td3_trainer.policy_net.get_action(state, deterministic = DETERMINISTIC, explore_noise_scale=explore_noise_scale)
    else:
      action = td3_trainer.policy_net.sample_action()
    next_state, reward, done, _ = env.step(action) 
                    # env.render()

    replay_buffer.push(state, action, reward, next_state, done)
                
    state = next_state
    episode_reward += reward
    frame_idx += 1
                
    if len(replay_buffer) > batch_size:
      for i in range(update_itr):
         loss = td3_trainer.update(batch_size, deterministic=DETERMINISTIC, eval_noise_scale=eval_noise_scale, reward_scale=reward_scale)
         episode_q_loss.append(float(loss['q_loss']))

         if 'policy_loss' in loss.keys():
           if loss['policy_loss'] != None:
              episode_policy_loss.append(loss['policy_loss'])

    if done:
      break
              
  if eps % 20 == 0 and eps>0:
    print('Episode: ', eps, '| Episode Reward: ', episode_reward)

            
  rewards.append(episode_reward)
  q_loss.append(np.mean(episode_q_loss))
  policy_loss.append(np.mean(episode_policy_loss))
  
        

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Episode:  20 | Episode Reward:  -648.2067633080514
Episode:  40 | Episode Reward:  -385.8550578753722
Episode:  60 | Episode Reward:  -389.1028207294071
Episode:  80 | Episode Reward:  -379.310374457832
Episode:  100 | Episode Reward:  -368.08934038140234
Episode:  120 | Episode Reward:  -127.14821661349694
Episode:  140 | Episode Reward:  -374.26051081478687
Episode:  160 | Episode Reward:  -488.25642586183073
Episode:  180 | Episode Reward:  -121.98097646891912
Episode:  200 | Episode Reward:  -1.117347510661071
Episode:  220 | Episode Reward:  -250.19627897461945
Episode:  240 | Episode Reward:  -253.3106016976357
Episode:  260 | Episode Reward:  -1.0975818847903263
Episode:  280 | Episode Reward:  -255.77557663326854
Episode:  300 | Episode Reward:  -249.84896573532546
Episode:  320 | Episode Reward:  -488.47289876488236
Episode:  340 | Episode Reward:  -0.6106746390025107
Episode:  360 | Episode Reward:  -408.05292869325143
Episode:  380 | Episode Reward:  -125.65859311474618
Epis

## Plot the curves

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py

In [ ]:
data = [go.Scatter(x=np.arange(len(rewards)), y=rewards, mode='lines')]
layout = go.Layout(title='Reward curve for TD3 Agent with stochastic policy - Pendulum-v0', xaxis_title='No. of episodes', yaxis_title='Scores', width=900, height=1200)
figure = go.Figure(data=data, layout=layout)
py.iplot(figure)

In [ ]:
from plotly.subplots import make_subplots
# Plot the loss curves
x_axis1 = np.arange(len(q_loss))
x_axis2 = np.arange(len(policy_loss))

fig = make_subplots(rows=1, cols=2, subplot_titles=('Q Loss', 'Policy_loss'))
q_loss_curve = go.Scatter(x=x_axis1, y=q_loss, mode='lines')
policy_loss_curve = go.Scatter(x=x_axis2, y=policy_loss, mode='lines')

fig.append_trace(q_loss_curve, 1, 1)
fig.append_trace(policy_loss_curve, 1, 2)

fig.update_xaxes(title_text='Episodes', row=1, col=1)
fig.update_xaxes(title_text='Episodes', row=1, col=2)

fig.update_yaxes(title_text='Q Loss', row=1, col=1)
fig.update_yaxes(title_text='Policy Loss', row=1, col=2)

fig.update_layout(title='Loss curves for TD3 Agent with stochastic policy')

py.iplot(fig)

## Comparison against Deterministic policy

In [ ]:
import pandas as pd

rewards_data = pd.read_csv('Rewards_data.csv')
loss_data = pd.read_csv('Loss_data.csv')

In [ ]:
loss_data.columns

Index(['Unnamed: 0', 'TD3_det_q', 'TD3_det_policy'], dtype='object')

In [ ]:
q_loss = q_loss[1:]
policy_loss = policy_loss[1:]

In [ ]:
len(list(loss_data['TD3_det_policy'])) == len(policy_loss)

True

In [ ]:
q_loss_det = list(loss_data['TD3_det_q'])
policy_loss_det = list(loss_data['TD3_det_policy'])

In [ ]:
# Compare between the traces
episodes = np.arange(len(q_loss))

fig = make_subplots(rows=1, cols=2, subplot_titles=('Q Loss', 'Policy_loss'))
q_loss_curve_det = go.Scatter(x=episodes, y=q_loss_det, mode='lines', name='deterministic')
policy_loss_curve_det = go.Scatter(x=episodes, y=policy_loss_det, mode='lines', name='deterministic')
q_loss_curve_stoc = go.Scatter(x=episodes, y=q_loss, mode='lines', name='stochastic')
policy_loss_curve_stoc = go.Scatter(x=episodes, y=policy_loss, mode='lines', name='stochastic')

q_curve = [q_loss_curve_det, q_loss_curve_stoc]
policy_curve = [policy_loss_curve_det, policy_loss_curve_stoc]

fig.add_traces(q_curve, rows=[1, 1], cols=[1, 1])
fig.add_traces(policy_curve, rows=[1, 1], cols=[2, 2])

fig.update_xaxes(title_text='Episodes', row=1, col=1)
fig.update_xaxes(title_text='Episodes', row=1, col=2)

fig.update_yaxes(title_text='Q Loss', row=1, col=1)
fig.update_yaxes(title_text='Policy Loss', row=1, col=2)

fig.update_layout(title='Loss curves for TD3 Agent on Pendulum-v0')

py.iplot(fig)

In [ ]:
len(list(rewards_data['TD3_det_agent'])) == len(rewards)

True

In [ ]:
# Plot rewards
episodes = np.arange(len(rewards))
reward_det = go.Scatter(x=episodes, y=list(rewards_data['TD3_det_agent']), mode='lines', name='deterministic')
reward_stoc = go.Scatter(x=episodes, y=rewards, mode='lines', name='stochastic')

reward_data = [reward_det, reward_stoc]

layout = go.Layout(title='Rewards by TD3 Agent on Pendulum-v0', xaxis_title='Episodes', yaxis_title='Rewards', width=1200, height=900)

figure = go.Figure(data=reward_data, layout=layout)

py.iplot(figure)

In [ ]:
# save the data
loss_data['TD3_stoc_q'] = q_loss
loss_data['TD3_stoc_policy'] = policy_loss

rewards_data['TD3_stoc_agent'] = rewards

In [ ]:
# Write the files
rewards_data.to_csv('Rewards.csv')
loss_data.to_csv('Loss.csv')